<a href="https://colab.research.google.com/github/borundev/pytorch_examples/blob/master/ClassificationWithUncertainity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --no-cache-dir -Uqq git+https://github.com/borundev/pytorch_examples.git@classification_with_uncertainity

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
import names

In [ ]:
import pytorch_examples

In [ ]:
from pytorch_examples.classification_with_uncertainity.data import CustomDataModule
from pytorch_examples.classification_with_uncertainity.models.resnet import CustomModel

In [ ]:
from pytorch_lightning.loggers import WandbLogger


In [ ]:
def run(num_extras):
  name='{}_{}'.format(num_extras,'_'.join(names.get_full_name().split()))
  wandb_logger = WandbLogger(project='uncertain_classification',name=name)
  trainer = pl.Trainer(
        gpus=1,
        max_epochs=10,
        logger=wandb_logger,
        progress_bar_refresh_rate=50,
        )
  model = CustomModel()
  cdm = CustomDataModule(num_extras=num_extras)
  trainer.fit(model, cdm)
  wandb.finish()

In [ ]:
for num_extras in (0,1,2,3,4):
  run(num_extras)

In [ ]:
x,y_original,y,n=next(iter(cdm.train_dataloader()))

In [ ]:
import torchvision.transforms as transforms


In [ ]:
n=transforms.Normalize((-1., -1.,- 1.), (2., 2., 2.))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x=n(x).permute(0,2,3,1)

In [ ]:
plt.imshow(x[2])

In [ ]:
import numpy as np
import torchvision.utils as vutils

dataloader = cdm.train_dataloader()
real_batch = next(iter(dataloader))
real_images = np.transpose(vutils.make_grid(real_batch[0][:6], padding=2, normalize=True).numpy(), (1, 2, 0))


In [ ]:
plt.imshow(real_images)

In [ ]:
real_batch[1:]

In [ ]:
from torch.utils.data import random_split, DataLoader


In [ ]:
DataLoader?

In [ ]:
!pip install names

In [ ]:
import names

In [ ]:
'_'.join(names.get_full_name().split())